In [1]:
import re

def text_clean(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    
    # pattern = '[^\w\s]'         # 특수기호제거
    # text = re.sub(pattern, '', text)
    
    pattern = '(?<!\d)-|\b(?!\d+-\d)\d+\b|[^\w\s-]' # 번지수
    text = re.sub(pattern, '', text)
    
    pattern = '\s{2, }'           # 중복 공백 제거
    text = re.sub(pattern, '', text).strip() 
    
    pattern = '\n'            # 줄 바꿈 제거
    text = re.sub(pattern, '', text) 

    # pattern = '\d+'        # 숫자 제거
    # text = re.sub(pattern, '', text) 

    return text 

In [2]:
import pandas as pd

df = pd.read_excel('/home/yjtech2/Desktop/yurim/LLM/Data/text_test_data.xlsx')

In [3]:
df['text'] = df['text'].apply(text_clean)

df

,text
0,음식물 쓰레기 냄세가 나요
1,전라북도 순창군 면 마을 꼴목길에 퇴비를 가져다 놓음바람이불면 골목안쪽으로 냄새가...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다발리조치해 주시기 바랍니다
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다
4,동국대 신공학관 근처 하수 처리 냄세 신고
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요원룸형 아파트 인데요 옆집...
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다
7,공공시설 길거리 흡연자가 녀무 많습니다
8,벌래와 모기 하수구냄새가 납니다
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다


In [4]:
import json
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import pandas as pd
import re
from difflib import SequenceMatcher
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# 모델 정의
tokenizer = AutoTokenizer.from_pretrained("fiveflow/roberta-base-spacing")
model = AutoModelForTokenClassification.from_pretrained("fiveflow/roberta-base-spacing")

# GPU 사용 가능시 GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 레이블 정의
label = ["UNK", "PAD", "O", "B", "I", "E", "S"]

def apply_spacing_correction(text):
    if not isinstance(text, str):
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 512 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 512 토큰 단위로 나누기
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            token_list = [tokenizer.cls_token_id]
            for char in chunk:
                token_list.extend(tokenizer.encode(char)[1:-1])  # CLS, SEP 토큰 제외
            token_list.append(tokenizer.sep_token_id)
            
            tkd = torch.tensor(token_list).unsqueeze(0).to(device)
            
            # 모델 예측
            with torch.no_grad():
                outputs = model(tkd)
                pred_idx = torch.argmax(outputs.logits, dim=-1)
            
            # 예측 결과를 문자열로 변환
            pred_sent = ""
            for char_idx, spc_idx in enumerate(pred_idx.squeeze()[1:-1]):
                if char_idx >= len(chunk):  # 인덱스 범위 체크
                    break
                
                curr_label = label[spc_idx]
                if curr_label in ["E", "S"]:  # E나 S 태그가 있으면 띄어쓰기 추가
                    pred_sent += chunk[char_idx] + " "
                else:
                    pred_sent += chunk[char_idx]
            
            corrected_sentences.append(pred_sent.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip()
    
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return text  # 에러 발생시 원본 텍스트 반환

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 띄어쓰기 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(apply_spacing_correction)
    
    return df

     
# 수행
if __name__ == "__main__":
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Start ======')
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    print(f'[{_now_time}] ====== Data Preprocessing Start ======')
    processed_df = process_dataframe(df, "text")
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Preprocessing Finished ======')
    
    def text_clean(text):
        # pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        # text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)


/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-19 13:20:31.397466: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 13:20:31.421317: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 13:20:31.785480: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

[2024-11-19 13:20:33.632480] ====== Data Load Start ======
[2024-11-19 13:20:33.632548] ====== Data Load Finished ======
[2024-11-19 13:20:33.632548] ====== Data Preprocessing Start ======


Processing spacing correction: 100%|██████████| 10/10 [00:00<00:00, 108.40it/s]

[2024-11-19 13:20:33.727519] ====== Data Preprocessing Finished ======


In [5]:
processed_df

,text,res_sentence
0,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄세가 나요
1,전라북도 순창군 면 마을 꼴목길에 퇴비를 가져다 놓음바람이불면 골목안쪽으로 냄새가...,전라북도 순창군 면마을 꼴목길에 퇴비를 가져다 놓음 바람이 불면 골목 안쪽으로 냄새...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다발리조치해 주시기 바랍니다,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다 발리 조치해 주시기 ...
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다
4,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄세 신고
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요원룸형 아파트 인데요 옆집...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸형 아파트인데요 ...
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다
7,공공시설 길거리 흡연자가 녀무 많습니다,공공시설 길거리 흡연자가 녀무 많습니다
8,벌래와 모기 하수구냄새가 납니다,벌래와 모기 하수구 냄새가 납니다
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다


## 기본적인 hunspell

In [7]:
import hunspell

print(dir(hunspell))

['HunSpell', 'HunSpellError', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'add', 'add_dic', 'add_with_affix', 'analyze', 'generate', 'generate2', 'get_dic_encoding', 'remove', 'spell', 'stem', 'suggest']


In [8]:
from hunspell import HunSpell

print(help(HunSpell))

Help on class HunSpell in module builtins:

class HunSpell(object)
 |  HunSpell binding. 
 |  
 |  Instantiation goes like this:
 |  >>> hobj = HunSpell('/path/to/dict.dic', '/path/to/dict.aff')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  add(...)
 |      Adds the given word into the runtime dictionary.
 |      
 |      Parameters
 |      ----------
 |      word : string
 |          The word to add in the dictionary
 |      
 |      Returns
 |      -------
 |      int : 0 if success, hunspell program error code else.
 |  
 |  add_dic(...)
 |      Load an extra dictionary to the current instance.
 |      The  extra dictionaries use the affix file of the allocated Hunspell object.
 |      Maximal number of the extra dictionaries is limited in the Hunspell source code to 20.
 |      
 |      Parameters
 |      ----------
 |      dpath : string
 |          Path to the .dic to add.

In [14]:
from hunspell import HunSpell
from datetime import datetime
import pandas as pd
import re
from tqdm import tqdm

def hunspell_basic(df, column_name):
    try:
        # Hunspell 객체 생성
        hunspell = HunSpell(
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.dic',
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.aff'
        )

        # 결과 저장용 리스트
        corrected_texts = []

        for text in df[column_name]:
            # 문장에서 단어 추출
            words = re.findall(r'\b\w+\b', text)
            
            # 수정된 문장을 저장할 변수
            corrected_sentence = text
            
            for word in words:
                if not hunspell.spell(word):
                    suggestions = hunspell.suggest(word)
                    
                    if suggestions:
                        corrected_word = suggestions[0]
                        corrected_sentence = re.sub(r'\b' + re.escape(word) + r'\b', corrected_word, corrected_sentence, count=1)
                        # print(f"'{word}'를 '{corrected_word}'로 수정했습니다.")
                    else:
                        print(f"'{word}'에 대한 제안이 없습니다.")
            
            corrected_texts.append(corrected_sentence)
            # print(f'\n수정 전 문장: {text}')
            # print(f'\n수정 후 문장: {corrected_sentence}')
        
        return corrected_texts

    except Exception as e:
        print(f"Hunspell 초기화 중 오류 발생: {e}")
        return []

if __name__ == "__main__":

    now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'[{now_time}] ====== hunspell Start ======')
    
    # 새로운 열에 수정된 결과 저장
    processed_df['hunspell_result'] = hunspell_basic(processed_df, 'res_sentence')
    
    now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f'[{now_time}] ====== hunspell End ======')


[2024-11-19 13:28:58] ====== hunspell Start ======
[2024-11-19 13:28:59] ====== hunspell End ======


In [20]:
processed_df

,text,res_sentence,hunspell_result
0,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄세가 나요,음식물 쓰레기 냄새가 나요
1,전라북도 순창군 면 마을 꼴목길에 퇴비를 가져다 놓음바람이불면 골목안쪽으로 냄새가...,전라북도 순창군 면마을 꼴목길에 퇴비를 가져다 놓음 바람이 불면 골목 안쪽으로 냄새...,전라북도 순차군 면 마을 골목길에 퇴비를 가져다 놓음 바람이 불면 골목 안쪽으로 냄...
2,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다발리조치해 주시기 바랍니다,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다 발리 조치해 주시기 ...,인도에 폐기물 컨테이너가 있어 통행 불편 및 악취로 불편합니다 발이 조치해 주시기 ...
3,이웃집 에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 쥽니다,이웃집에서 보일러 기름으로 추정되는 물질이 다량으로 유출되어 주변에 피해를 잡니다
4,동국대 신공학관 근처 하수 처리 냄세 신고,동국대 신공학관 근처 하수 처리 냄세 신고,동국도 신 공학관 근처 하수 처리 냄새 신고
5,층간소음 문제로 12년된 아파트 하자 점검 서비스가 있나요원룸형 아파트 인데요 옆집...,층간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸형 아파트인데요 ...,증간 소음 문제로 12년 된 아파트 하자 점검 서비스가 있나요 원룸 형 아파트인데요...
6,건물 옆 주차장에 길고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다,건물 옆 주차장에 길 고양들이 배설물을 싸고 있습니다
7,공공시설 길거리 흡연자가 녀무 많습니다,공공시설 길거리 흡연자가 녀무 많습니다,공공시설 길거리 흡연자가 누며 많습니다
8,벌래와 모기 하수구냄새가 납니다,벌래와 모기 하수구 냄새가 납니다,벌레와 모기 하수구 냄새가 납니다
9,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서족에서 불고 불쾌한 냄새가 납니다,바람이 동서 족에서 불고 불쾌한 냄새가 납니다


In [16]:
processed_df.to_excel('hunspell_result.xlsx')

## Cosine Similarity 계산 이용 

In [27]:
from hunspell import HunSpell
from datetime import datetime
import pandas as pd
import re
from tqdm import tqdm


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cosine Similarity 계산을 위한 함수
def calculate_similarity(word1, word2):
    vectorizer = TfidfVectorizer().fit_transform([word1, word2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

try:
    # Hunspell 객체 생성
    hunspell = HunSpell(
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.dic',
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.aff'

    )

    # 맞춤법 검사할 텍스트
    text = "바람이 남서방향족에서 불고 불쾌한 냄새가 납니다."
    
    # 문장을 단락으로 나누기
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    
    # 수정된 문장을 저장할 변수
    corrected_sentence = []
    
    # 앞 문장을 저장할 변수
    previous_sentence = ""

    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence)
        
        for word in words:
            if not hunspell.spell(word):
                suggestions = hunspell.suggest(word)
                if suggestions:
                    # 제안된 단어와 유사도 값을 계산
                    similarity_scores = [(suggestion, calculate_similarity(previous_sentence + " " + sentence, previous_sentence + " " + sentence.replace(word, suggestion))) for suggestion in suggestions]
                    
                    # 유사도 값을 기준으로 상위 3개의 제안을 선택
                    top_3_suggestions = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[:3]
                    
                    # 선택된 제안 중에서 유사도가 가장 높은 제안을 선택
                    best_suggestion = top_3_suggestions[0][0]
                    sentence = sentence.replace(word, best_suggestion, 1)
                    
                    print(f"\n단어 '{word}'에 대한 상위 3개 제안 및 유사도:")
                    for suggestion, score in top_3_suggestions:
                        print(f"제안: {suggestion}, 유사도: {score:.4f}")
                    
                    print(f"'{word}'를 '{best_suggestion}'로 수정했습니다.")
                else:
                    print(f"'{word}'에 대한 제안이 없습니다.")
        
        corrected_sentence.append(sentence)
        previous_sentence = sentence
    print('previous_sentence: ', previous_sentence)
    
    # 교정된 문장 출력
    final_text = ' '.join(corrected_sentence)
    print(f'\n수정 전 문장: {text}') 
    print(f'\n수정 후 문장: {final_text}')
    
except Exception as e:
    print(f"Hunspell 초기화 중 오류 발생: {e}")


단어 '남서방향족에서'에 대한 상위 3개 제안 및 유사도:
제안: 조선왕조실록에서, 유사도: 0.7168
'남서방향족에서'를 '조선왕조실록에서'로 수정했습니다.
previous_sentence:  바람이 조선왕조실록에서 불고 불쾌한 냄새가 납니다.

수정 전 문장: 바람이 남서방향족에서 불고 불쾌한 냄새가 납니다.

수정 후 문장: 바람이 조선왕조실록에서 불고 불쾌한 냄새가 납니다.


## 형태소 분석 + n-gram

In [2]:
from konlpy.tag import Komoran  # Komoran으로 변경
from hunspell import HunSpell
from datetime import datetime
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import os

class EnhancedSpellChecker:
    def __init__(self, dict_path, aff_path, ngram_corpus_path=None):
        """
        Enhanced spell checker initialization
        Args:
            dict_path: Path to Hunspell dictionary
            aff_path: Path to Hunspell affix file
            ngram_corpus_path: Path to pre-trained n-gram corpus (optional)
        """
        self.hunspell = HunSpell(dict_path, aff_path)
        self.komoran = Komoran()  # Komoran 초기화
        self.vectorizer = TfidfVectorizer(
            analyzer='char',
            ngram_range=(2, 3),  # 2-gram과 3-gram 모두 사용
            min_df=2
        )
        self.ngram_model = self._load_or_create_ngram_model(ngram_corpus_path)
        
    def _load_or_create_ngram_model(self, corpus_path):
        """N-gram 모델 로드 또는 생성"""
        if corpus_path and os.path.exists(corpus_path):
            with open(corpus_path, 'rb') as f:
                return pickle.load(f)
        return defaultdict(lambda: defaultdict(int))

    def _calculate_contextual_similarity(self, original_context, candidate_context):
        """문맥 기반 유사도 계산"""
        # 형태소 분석 결과를 활용한 유사도 계산
        orig_morphs = ' '.join(self.komoran.morphs(original_context))  # Komoran으로 변경
        cand_morphs = ' '.join(self.komoran.morphs(candidate_context))  # Komoran으로 변경
        
        # TF-IDF 벡터화
        vectors = self.vectorizer.fit_transform([orig_morphs, cand_morphs])
        
        # 코사인 유사도 계산
        return cosine_similarity(vectors)[0, 1]

    def _calculate_phonetic_similarity(self, word1, word2):
        """음소 기반 유사도 계산"""
        # 초성, 중성, 종성으로 분리하여 비교
        def decompose_hangul(text):
            CHOSUNG = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            JUNGSUNG = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
            JONGSUNG = [' ','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            
            result = []
            for char in text:
                if '가' <= char <= '힣':
                    char_code = ord(char) - ord('가')
                    jong = char_code % 28
                    jung = ((char_code - jong) // 28) % 21
                    cho = (((char_code - jong) // 28) - jung) // 21
                    result.extend([CHOSUNG[cho], JUNGSUNG[jung], JONGSUNG[jong]])
            return result

        # 자모 단위로 분해하여 유사도 계산
        char1 = decompose_hangul(word1)
        char2 = decompose_hangul(word2)
        
        # Levenshtein distance 계산
        def levenshtein_distance(s1, s2):
            if len(s1) < len(s2):
                return levenshtein_distance(s2, s1)
            if len(s2) == 0:
                return len(s1)
            
            previous_row = range(len(s2) + 1)
            for i, c1 in enumerate(s1):
                current_row = [i + 1]
                for j, c2 in enumerate(s2):
                    insertions = previous_row[j + 1] + 1
                    deletions = current_row[j] + 1
                    substitutions = previous_row[j] + (c1 != c2)
                    current_row.append(min(insertions, deletions, substitutions))
                previous_row = current_row
            
            return previous_row[-1]
        
        distance = levenshtein_distance(char1, char2)
        max_len = max(len(char1), len(char2))
        return 1 - (distance / max_len)

    def _get_window_context(self, words, current_idx, window_size=2):
        """주변 단어들의 문맥 추출"""
        start = max(0, current_idx - window_size)
        end = min(len(words), current_idx + window_size + 1)
        return ' '.join(words[start:end])

    def correct_text(self, text, min_similarity=0.6):
        """텍스트 맞춤법 교정"""
        # 문장 분리
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        corrected_sentences = []
        
        for sentence in sentences:
            words = re.findall(r'\b\w+\b', sentence)
            corrected_words = words.copy()
            
            for i, word in enumerate(words):
                if not self.hunspell.spell(word):
                    suggestions = self.hunspell.suggest(word)
                    if suggestions:
                        # 문맥 추출
                        context = self._get_window_context(words, i)
                        
                        # 각 제안에 대한 종합 점수 계산
                        scored_suggestions = []
                        for suggestion in suggestions:
                            # 새로운 문맥 생성
                            new_context = context.replace(word, suggestion)
                            
                            # 다양한 유사도 점수 계산
                            contextual_score = self._calculate_contextual_similarity(context, new_context)
                            phonetic_score = self._calculate_phonetic_similarity(word, suggestion)
                            
                            # N-gram 확률 계산
                            ngram_score = self._calculate_ngram_probability(new_context)
                            
                            # 종합 점수 계산 (가중치 적용)
                            total_score = (
                                contextual_score * 0.4 +
                                phonetic_score * 0.3 +
                                ngram_score * 0.3
                            )
                            
                            scored_suggestions.append((suggestion, total_score))
                        
                        # 가장 높은 점수의 제안 선택
                        scored_suggestions.sort(key=lambda x: x[1], reverse=True)
                        best_suggestion, best_score = scored_suggestions[0]
                        
                        # 최소 유사도 threshold를 넘는 경우에만 수정
                        if best_score >= min_similarity:
                            corrected_words[i] = best_suggestion
                            print(f"\n단어 '{word}' 교정:")
                            print(f"선택된 교정: {best_suggestion} (점수: {best_score:.4f})")
                            for sugg, score in scored_suggestions[:3]:
                                print(f"- 후보: {sugg}, 점수: {score:.4f}")
            
            corrected_sentence = ' '.join(corrected_words)
            corrected_sentences.append(corrected_sentence)
        
        return ' '.join(corrected_sentences)

    def _calculate_ngram_probability(self, context):
        """N-gram 확률 계산"""
        words = context.split()
        if len(words) < 2:
            return 0.5  # 기본값
        
        total_prob = 0
        count = 0
        
        for i in range(len(words) - 1):
            bigram = tuple(words[i:i+2])
            if bigram in self.ngram_model:
                total_prob += self.ngram_model[bigram] / sum(self.ngram_model[bigram[0]].values())
                count += 1
        
        return total_prob / max(1, count)

    def train_ngram_model(self, corpus_text):
        """N-gram 모델 학습"""
        sentences = re.split(r'(?<=[.!?])\s+', corpus_text.strip())
        for sentence in sentences:
            words = sentence.split()
            for i in range(len(words) - 1):
                self.ngram_model[words[i]][words[i+1]] += 1

# 사용 예시
if __name__ == "__main__":
    checker = EnhancedSpellChecker(
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.dic',
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.aff'
    )
    
    # 테스트 텍스트
    test_text = "바람이 남서방향족에서 불고 불쾌한 냄새가 납니다."
    
    # 맞춤법 교정 실행
    corrected_text = checker.correct_text(test_text)
    
    print(f"\n원본 텍스트: {test_text}")
    print(f"교정된 텍스트: {corrected_text}")



원본 텍스트: 바람이 남서방향족에서 불고 불쾌한 냄새가 납니다.
교정된 텍스트: 바람이 남서방향족에서 불고 불쾌한 냄새가 납니다


## 형태소 분석 + n-gram 확장 + 음소 기반 유사도 개선

In [6]:
from konlpy.tag import Okt  # Okt로 변경
from hunspell import HunSpell
from datetime import datetime
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import os

class EnhancedSpellChecker:
    def __init__(self, dict_path, aff_path, ngram_corpus_path=None):
        """
        Enhanced spell checker initialization
        Args:
            dict_path: Path to Hunspell dictionary
            aff_path: Path to Hunspell affix file
            ngram_corpus_path: Path to pre-trained n-gram corpus (optional)
        """
        self.hunspell = HunSpell(dict_path, aff_path)
        self.okt = Okt()  # Okt 초기화
        self.vectorizer = TfidfVectorizer(
            analyzer='char',
            ngram_range=(2, 3),  # 2-gram과 3-gram 모두 사용
            min_df=2
        )
        self.ngram_model = self._load_or_create_ngram_model(ngram_corpus_path)
        
    def _load_or_create_ngram_model(self, corpus_path):
        """N-gram 모델 로드 또는 생성"""
        if corpus_path and os.path.exists(corpus_path):
            with open(corpus_path, 'rb') as f:
                return pickle.load(f)
        return defaultdict(lambda: defaultdict(int))

    def _calculate_contextual_similarity(self, original_context, candidate_context):
        """문맥 기반 유사도 계산"""
        # 형태소 분석 결과를 활용한 유사도 계산
        orig_morphs = ' '.join(self.okt.morphs(original_context))  # Okt로 변경
        cand_morphs = ' '.join(self.okt.morphs(candidate_context))  # Okt로 변경
        
        # TF-IDF 벡터화
        vectors = self.vectorizer.fit_transform([orig_morphs, cand_morphs])
        
        # 코사인 유사도 계산
        return cosine_similarity(vectors)[0, 1]

    def _calculate_phonetic_similarity(self, word1, word2):
        """음소 기반 유사도 계산"""
        # 초성, 중성, 종성으로 분리하여 비교
        def decompose_hangul(text):
            CHOSUNG = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            JUNGSUNG = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
            JONGSUNG = [' ','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            
            result = []
            for char in text:
                if '가' <= char <= '힣':
                    char_code = ord(char) - ord('가')
                    jong = char_code % 28
                    jung = ((char_code - jong) // 28) % 21
                    cho = (((char_code - jong) // 28) - jung) // 21
                    result.extend([CHOSUNG[cho], JUNGSUNG[jung], JONGSUNG[jong]])
            return result

        # 자모 단위로 분해하여 유사도 계산
        char1 = decompose_hangul(word1)
        char2 = decompose_hangul(word2)
        
        # Levenshtein distance 계산
        def levenshtein_distance(s1, s2):
            if len(s1) < len(s2):
                return levenshtein_distance(s2, s1)
            if len(s2) == 0:
                return len(s1)
            
            previous_row = range(len(s2) + 1)
            for i, c1 in enumerate(s1):
                current_row = [i + 1]
                for j, c2 in enumerate(s2):
                    insertions = previous_row[j + 1] + 1
                    deletions = current_row[j] + 1
                    substitutions = previous_row[j] + (c1 != c2)
                    current_row.append(min(insertions, deletions, substitutions))
                previous_row = current_row
            
            return previous_row[-1]
        
        distance = levenshtein_distance(char1, char2)
        max_len = max(len(char1), len(char2))
        return 1 - (distance / max_len)

    def _get_window_context(self, words, current_idx, window_size=2):
        """주변 단어들의 문맥 추출"""
        start = max(0, current_idx - window_size)
        end = min(len(words), current_idx + window_size + 1)
        return ' '.join(words[start:end])

    def correct_text(self, text, min_similarity=0.6):
        """텍스트 맞춤법 교정"""
        # 문장 분리
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        corrected_sentences = []
        
        for sentence in sentences:
            words = re.findall(r'\b\w+\b', sentence)
            corrected_words = words.copy()
            
            for i, word in enumerate(words):
                if not self.hunspell.spell(word):
                    suggestions = self.hunspell.suggest(word)
                    if suggestions:
                        # 문맥 추출
                        context = self._get_window_context(words, i)
                        
                        # 각 제안에 대한 종합 점수 계산
                        scored_suggestions = []
                        for suggestion in suggestions:
                            # 새로운 문맥 생성
                            new_context = context.replace(word, suggestion)
                            
                            # 다양한 유사도 점수 계산
                            contextual_score = self._calculate_contextual_similarity(context, new_context)
                            phonetic_score = self._calculate_phonetic_similarity(word, suggestion)
                            
                            # N-gram 확률 계산
                            ngram_score = self._calculate_ngram_probability(new_context)
                            
                            # 종합 점수 계산 (가중치 적용)
                            total_score = (
                                contextual_score * 0.4 +
                                phonetic_score * 0.3 +
                                ngram_score * 0.3
                            )
                            
                            scored_suggestions.append((suggestion, total_score))
                        
                        # 가장 높은 점수의 제안 선택
                        scored_suggestions.sort(key=lambda x: x[1], reverse=True)
                        best_suggestion, best_score = scored_suggestions[0]
                        
                        # 최소 유사도 threshold를 넘는 경우에만 수정
                        if best_score >= min_similarity:
                            corrected_words[i] = best_suggestion
                            print(f"\n단어 '{word}' 교정:")
                            print(f"선택된 교정: {best_suggestion} (점수: {best_score:.4f})")
                            for sugg, score in scored_suggestions[:3]:
                                print(f"- 후보: {sugg}, 점수: {score:.4f}")
            
            corrected_sentence = ' '.join(corrected_words)
            corrected_sentences.append(corrected_sentence)
        
        return ' '.join(corrected_sentences)

    def _calculate_ngram_probability(self, context):
        """N-gram 확률 계산"""
        words = context.split()
        if len(words) < 2:
            return 0.5  # 기본값

        total_prob = 0
        count = 0

        # Bigram 확률 계산
        for i in range(len(words) - 1):
            bigram = tuple(words[i:i+2])
            if bigram in self.ngram_model:
                total_prob += self.ngram_model[bigram[0]][bigram[1]] / sum(self.ngram_model[bigram[0]].values())
                count += 1

        # Trigram 확률 계산
        for i in range(len(words) - 2):
            trigram = tuple(words[i:i+3])
            if (
                trigram[0] in self.ngram_model
                and trigram[1] in self.ngram_model[trigram[0]]
                and trigram[2] in self.ngram_model[trigram[0]][trigram[1]]
            ):
                total_prob += self.ngram_model[trigram[0]][trigram[1]][trigram[2]] / sum(
                    self.ngram_model[trigram[0]][trigram[1]].values()
                )
                count += 1

        return total_prob / max(1, count)

    def train_ngram_model(self, corpus_text):
        """N-gram 모델 학습"""
        sentences = re.split(r'(?<=[.!?])\s+', corpus_text.strip())
        for sentence in sentences:
            words = sentence.split()
            # Bigram 학습
            for i in range(len(words) - 1):
                bigram = tuple(words[i:i+2])
                self.ngram_model[bigram[0]][bigram[1]] += 1
            # Trigram 학습
            for i in range(len(words) - 2):
                trigram = tuple(words[i:i+3])
                if trigram[0] not in self.ngram_model:
                    self.ngram_model[trigram[0]] = defaultdict(lambda: defaultdict(int))
                self.ngram_model[trigram[0]][trigram[1]][trigram[2]] += 1

# 사용 예시
if __name__ == "__main__":
    checker = EnhancedSpellChecker(
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.dic',
        '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko_KR.aff'
    )
    
    # 테스트 텍스트
    test_text = "바람이 어디에서 부는진 모르겠지만 이샹한 음식물 스레기 냄새가 난다"
    
    # 맞춤법 교정 실행
    corrected_text = checker.correct_text(test_text)
    
    print(f"\n원본 텍스트: {test_text}")
    print(f"교정된 텍스트: {corrected_text}")



단어 '이샹한' 교정:
선택된 교정: 이상한 (점수: 0.6667)
- 후보: 이상한, 점수: 0.6667
- 후보: 이송한, 점수: 0.6667
- 후보: 이양한, 점수: 0.6667

단어 '스레기' 교정:
선택된 교정: 쓰레기 (점수: 0.6667)
- 후보: 쓰레기, 점수: 0.6667
- 후보: 수레기, 점수: 0.6667

원본 텍스트: 바람이 어디에서 부는진 모르겠지만 이샹한 음식물 스레기 냄새가 난다
교정된 텍스트: 바람이 어디에서 부는진 모르겠지만 이상한 음식물 쓰레기 냄새가 난다


## 성능 테스트

In [1]:
from konlpy.tag import Okt  # Okt로 변경
import json
from hunspell import HunSpell
from datetime import datetime
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# 데이터 불러오기
def load_data(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'annotation' in item:
            annotation = item['annotation']
            
            if 'err_sentence' in annotation and 'cor_sentence' in annotation:
                data.append({
                    'err_sentence': annotation['err_sentence'],
                    'cor_sentence': annotation['cor_sentence']
                })
                
    df = pd.DataFrame(data)
    return df


class EnhancedSpellChecker:
    def __init__(self, dict_path, aff_path, ngram_corpus_path=None):
        """
        Enhanced spell checker initialization
        Args:
            dict_path: Path to Hunspell dictionary
            aff_path: Path to Hunspell affix file
            ngram_corpus_path: Path to pre-trained n-gram corpus (optional)
        """
        self.hunspell = HunSpell(dict_path, aff_path)
        self.okt = Okt()  # Okt 초기화
        self.vectorizer = TfidfVectorizer(
            analyzer='char',
            ngram_range=(2, 3),  # 2-gram과 3-gram 모두 사용
            min_df=2
        )
        self.ngram_model = self._load_or_create_ngram_model(ngram_corpus_path)
        
    def _load_or_create_ngram_model(self, corpus_path):
        """N-gram 모델 로드 또는 생성"""
        if corpus_path and os.path.exists(corpus_path):
            with open(corpus_path, 'rb') as f:
                return pickle.load(f)
        return defaultdict(lambda: defaultdict(int))

    def _calculate_contextual_similarity(self, original_context, candidate_context):
        """문맥 기반 유사도 계산"""
        # 형태소 분석 결과를 활용한 유사도 계산
        orig_morphs = ' '.join(self.okt.morphs(original_context))  # Okt로 변경
        cand_morphs = ' '.join(self.okt.morphs(candidate_context))  # Okt로 변경
        
        # TF-IDF 벡터화
        vectors = self.vectorizer.fit_transform([orig_morphs, cand_morphs])
        
        # 코사인 유사도 계산
        return cosine_similarity(vectors)[0, 1]

    def _calculate_phonetic_similarity(self, word1, word2):
        """음소 기반 유사도 계산"""
        # 초성, 중성, 종성으로 분리하여 비교
        def decompose_hangul(text):
            CHOSUNG = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            JUNGSUNG = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
            JONGSUNG = [' ','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
            
            result = []
            for char in text:
                if '가' <= char <= '힣':
                    char_code = ord(char) - ord('가')
                    jong = char_code % 28
                    jung = ((char_code - jong) // 28) % 21
                    cho = (((char_code - jong) // 28) - jung) // 21
                    result.extend([CHOSUNG[cho], JUNGSUNG[jung], JONGSUNG[jong]])
            return result

        # 자모 단위로 분해하여 유사도 계산
        char1 = decompose_hangul(word1)
        char2 = decompose_hangul(word2)
        
        # Levenshtein distance 계산
        def levenshtein_distance(s1, s2):
            if len(s1) < len(s2):
                return levenshtein_distance(s2, s1)
            if len(s2) == 0:
                return len(s1)
            
            previous_row = range(len(s2) + 1)
            for i, c1 in enumerate(s1):
                current_row = [i + 1]
                for j, c2 in enumerate(s2):
                    insertions = previous_row[j + 1] + 1
                    deletions = current_row[j] + 1
                    substitutions = previous_row[j] + (c1 != c2)
                    current_row.append(min(insertions, deletions, substitutions))
                previous_row = current_row
            
            return previous_row[-1]
        
        distance = levenshtein_distance(char1, char2)
        max_len = max(len(char1), len(char2))
        return 1 - (distance / max_len)

    def _get_window_context(self, words, current_idx, window_size=2):
        """주변 단어들의 문맥 추출"""
        start = max(0, current_idx - window_size)
        end = min(len(words), current_idx + window_size + 1)
        return ' '.join(words[start:end])

    def correct_text(self, text, min_similarity=0.6):
        """텍스트 맞춤법 교정"""
        # 문장 분리
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        corrected_sentences = []
        
        for sentence in sentences:
            words = re.findall(r'\b\w+\b', sentence)
            corrected_words = words.copy()
            
            for i, word in enumerate(words):
                if not self.hunspell.spell(word):
                    suggestions = self.hunspell.suggest(word)
                    if suggestions:
                        # 문맥 추출
                        context = self._get_window_context(words, i)
                        
                        # 각 제안에 대한 종합 점수 계산
                        scored_suggestions = []
                        for suggestion in suggestions:
                            # 새로운 문맥 생성
                            new_context = context.replace(word, suggestion)
                            
                            # 다양한 유사도 점수 계산
                            contextual_score = self._calculate_contextual_similarity(context, new_context)
                            phonetic_score = self._calculate_phonetic_similarity(word, suggestion)
                            
                            # N-gram 확률 계산
                            ngram_score = self._calculate_ngram_probability(new_context)
                            
                            # 종합 점수 계산 (가중치 적용)
                            total_score = (
                                contextual_score * 0.4 +
                                phonetic_score * 0.3 +
                                ngram_score * 0.3
                            )
                            
                            scored_suggestions.append((suggestion, total_score))
                        
                        # 가장 높은 점수의 제안 선택
                        scored_suggestions.sort(key=lambda x: x[1], reverse=True)
                        best_suggestion, best_score = scored_suggestions[0]
                        
                        # 최소 유사도 threshold를 넘는 경우에만 수정
                        if best_score >= min_similarity:
                            corrected_words[i] = best_suggestion
                            # print(f"\n단어 '{word}' 교정:")
                            # print(f"선택된 교정: {best_suggestion} (점수: {best_score:.4f})")
                            # for sugg, score in scored_suggestions[:3]:
                            #     print(f"- 후보: {sugg}, 점수: {score:.4f}")
            
            corrected_sentence = ' '.join(corrected_words)
            corrected_sentences.append(corrected_sentence)
        
        return ' '.join(corrected_sentences)

    def _calculate_ngram_probability(self, context):
        """N-gram 확률 계산"""
        words = context.split()
        if len(words) < 2:
            return 0.5  # 기본값
        
        total_prob = 0
        count = 0
        
        for i in range(len(words) - 1):
            bigram = tuple(words[i:i+2])
            if bigram in self.ngram_model:
                total_prob += self.ngram_model[bigram] / sum(self.ngram_model[bigram[0]].values())
                count += 1
        
        return total_prob / max(1, count)

    def train_ngram_model(self, corpus_text):
        """N-gram 모델 학습"""
        sentences = re.split(r'(?<=[.!?])\s+', corpus_text.strip())
        for sentence in sentences:
            words = sentence.split()
            for i in range(len(words) - 1):
                self.ngram_model[words[i]][words[i+1]] += 1


    # 결과 확인
    def add_comparison_columns(df):
        
        def calculate_spacing_similarity(row):
            """
            띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
            """
            if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
                return 0
            
            # 문장의 띄어쓰기를 유지한 채로 비교
            res = row['res_sentence']
            cor = row['cor_sentence']
            
            # 완전히 같으면 1 반환
            if res == cor:
                return 1.0
            
            # 다르면 띄어쓰기를 포함한 유사도 계산
            similarity = SequenceMatcher(None, res, cor).ratio()
            
            # 띄어쓰기 패턴의 정확도를 별도로 계산
            res_spaces = [i for i, char in enumerate(res) if char == ' ']
            cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
            
            # 공통된 띄어쓰기 위치 수 계산
            common_spaces = len(set(res_spaces) & set(cor_spaces))
            total_spaces = len(set(res_spaces) | set(cor_spaces))
            
            # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
            spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
            
            # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
            final_ratio = (similarity + spacing_accuracy) / 2
            
            return round(final_ratio, 3)
        
        # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
        df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
        
        # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
        df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
        
        return df

# 수행

if __name__ == "__main__":
    # 시작 시간 기록
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Start ======')
    
    # 데이터 로드
    df = load_data('/home/yjtech2/Desktop/yurim/LLM/Data/맞춤법오류_자유게시판.json')
    df = df.iloc[:100]
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    # Spell Checker 초기화
    print(f'[{_now_time}] ====== Initializing Spell Checker ======')
    spell_checker = EnhancedSpellChecker(
        dict_path='/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko.dic',  # Hunspell 사전 파일 경로
        aff_path='/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko.aff',  # Hunspell affix 파일 경로
    )
    
    print(f'[{_now_time}] ====== Processing Text Correction ======')
    # 오류 문장에 대한 교정 수행
    processed_data = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        err_sentence = row['err_sentence']
        cor_sentence = row['cor_sentence']
        
        # 맞춤법 교정 수행
        res_sentence = spell_checker.correct_text(err_sentence)
        
        processed_data.append({
            'err_sentence': err_sentence,
            'res_sentence': res_sentence,
            'cor_sentence': cor_sentence
        })
    
    # DataFrame 생성
    processed_df = pd.DataFrame(processed_data)
    
    # 텍스트 클리닝 함수 정의
    def text_clean(text):
        pattern = '[^\w\s]'  # 특수기호제거
        text = re.sub(pattern, '', text)
        return text
    
    # 텍스트 클리닝 적용
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    # 비교 컬럼 추가
    result_df = spell_checker.add_comparison_columns(processed_df)
    
    # 결과 출력
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Evaluation Results ======')
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))
    
    # 결과 저장 (선택사항)
    result_df.to_csv('spell_check_results.csv', index=False, encoding='utf-8-sig')
    print(f'[{_now_time}] ====== Results Saved ======')

[2024-11-20 10:02:30.423909] ====== Data Load Start ======
[2024-11-20 10:02:30.580781] ====== Data Load Finished ======
[2024-11-20 10:02:30.580781] ====== Initializing Spell Checker ======
[2024-11-20 10:02:30.580781] ====== Processing Text Correction ======


100%|██████████| 100/100 [00:14<00:00,  7.03it/s]


TypeError: add_comparison_columns() takes 1 positional argument but 2 were given

In [7]:
import json
from hunspell import HunSpell
from tqdm import tqdm
import pandas as pd
import re
from datetime import datetime
from difflib import SequenceMatcher

# 데이터 불러오기
def load_data(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'annotation' in item:
            annotation = item['annotation']
            
            if 'err_sentence' in annotation and 'cor_sentence' in annotation:
                data.append({
                    'err_sentence': annotation['err_sentence'],
                    'cor_sentence': annotation['cor_sentence']
                    })
                
    df = pd.DataFrame(data)
    return df


def hunspell_basic(text):
    try:
        # Hunspell 객체 생성
        hunspell = HunSpell(
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko.dic',
            '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/spelling/hunspell/ko.aff'
        )

        # 결과 저장용 리스트
        corrected_texts = []

        # 문장에서 단어 추출
        words = re.findall(r'\b\w+\b', text)
        
        # 수정된 문장을 저장할 변수
        corrected_sentence = text
        
        for word in words:
            if not hunspell.spell(word):
                suggestions = hunspell.suggest(word)
                
                if suggestions:
                    corrected_word = suggestions[0]
                    corrected_sentence = re.sub(r'\b' + re.escape(word) + r'\b', corrected_word, corrected_sentence, count=1)
                    # print(f"'{word}'를 '{corrected_word}'로 수정했습니다.")
                else:
                    print(f"'{word}'에 대한 제안이 없습니다.")
        
        corrected_texts.append(corrected_sentence)
        # print(f'\n수정 전 문장: {text}')
        # print(f'\n수정 후 문장: {corrected_sentence}')
        
        return corrected_texts

    except Exception as e:
        print(f"Hunspell 초기화 중 오류 발생: {e}")
        return []

def process_dataframe(df, text_column):
    # 진행바 표시
    tqdm.pandas(desc="Processing spacing correction")
    
    # 맞춤법 교정 적용
    df["res_sentence"] = df[text_column].progress_apply(hunspell_basic)
    
    return df


# 결과 확인
def add_comparison_columns(df):
    
    def calculate_spacing_similarity(row):
        """
        띄어쓰기를 포함한 두 문장 간의 유사도를 계산하는 함수
        """
        if pd.isna(row['res_sentence']) or pd.isna(row['cor_sentence']):
            return 0
        
        # 문장의 띄어쓰기를 유지한 채로 비교
        res = row['res_sentence']
        cor = row['cor_sentence']
        
        # 완전히 같으면 1 반환
        if res == cor:
            return 1.0
        
        # 다르면 띄어쓰기를 포함한 유사도 계산
        similarity = SequenceMatcher(None, res, cor).ratio()
        
        # 띄어쓰기 패턴의 정확도를 별도로 계산
        res_spaces = [i for i, char in enumerate(res) if char == ' ']
        cor_spaces = [i for i, char in enumerate(cor) if char == ' ']
        
        # 공통된 띄어쓰기 위치 수 계산
        common_spaces = len(set(res_spaces) & set(cor_spaces))
        total_spaces = len(set(res_spaces) | set(cor_spaces))
        
        # 띄어쓰기 정확도 (공통 띄어쓰기 / 전체 띄어쓰기)
        spacing_accuracy = common_spaces / total_spaces if total_spaces > 0 else 1.0
        
        # 전체 유사도와 띄어쓰기 정확도를 조합 (가중치는 조정 가능)
        final_ratio = (similarity + spacing_accuracy) / 2
        
        return round(final_ratio, 3)
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가 (띄어쓰기를 고려한 유사도 계산)
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

# 수행
if __name__ == "__main__":
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Start ======')
    df = load_data('/home/yjtech2/Desktop/yurim/LLM/Data/맞춤법오류_자유게시판.json')
    df = df.iloc[:100]
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Load Finished ======')
    
    print(f'[{_now_time}] ====== Data Preprocessing Start ======')
    processed_df = process_dataframe(df, "err_sentence")
    _now_time = datetime.now().__str__()
    print(f'[{_now_time}] ====== Data Preprocessing Finished ======')

    processed_df = processed_df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    def text_clean(text):
        pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
        text = re.sub(pattern, '', text)
        
        pattern = '[^\w\s]'         # 특수기호제거
        text = re.sub(pattern, '', text)

        return text 
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(lambda x: x[0] if isinstance(x, list) else x)
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

[2024-11-20 10:04:58.333196] ====== Data Load Start ======
[2024-11-20 10:04:58.547831] ====== Data Load Finished ======
[2024-11-20 10:04:58.547831] ====== Data Preprocessing Start ======


Processing spacing correction:  24%|██▍       | 24/100 [00:07<00:27,  2.77it/s]

'코로나떄문에'에 대한 제안이 없습니다.


Processing spacing correction:  39%|███▉      | 39/100 [00:12<00:23,  2.58it/s]

'배배꼬아논거같타요'에 대한 제안이 없습니다.


...

'혐오한다햇엇음'에 대한 제안이 없습니다.


Processing spacing correction:  89%|████████▉ | 89/100 [00:29<00:03,  3.33it/s]

'수험생기만한건'에 대한 제안이 없습니다.


Processing spacing correction: 100%|██████████| 100/100 [00:32<00:00,  3.04it/s]

[2024-11-20 10:05:31.427945] ====== Data Preprocessing Finished ======
비교 완료
정확도:  0.07
비율 평균:  0.72099


In [69]:
result_df

,err_sentence,res_sentence,cor_sentence,check,ratio
0,오늘도 다너를 외우러 와따요,오늘도 단어를 외우러 왕따요,오늘도 단어를 외우러 왔어요,0,0.933
1,오늘은 추석 연휴동안 놀고 먹으며 학회 레포트를 쓰는 영상 준비해보았습니디,오늘은 추석 연휴 동안 놀고 먹으며 학회 리포트를 쓰는 영상 준비해보았습니다,오늘은 추석 연휴 동안 놀고먹으며 학회 리포트를 쓰는 영상 준비해 보았습니다,0,0.613
2,요즘 새로 살까말까 고민중인데 대학원 가면 새로 살것 같기도 하구 그러네요,요즘 새로 살 까말까 고 민중인데 대학원 가면 새로 살 것 같기도 하구 그러네요,요즘 새로 살까 말까 고민 중인데 대학원 가면 새로 살 것 같기도 하고 그러네요,0,0.833
3,요즘 영어 문제르 안 풀어서 실력이 내려가고 있는거 가타요.,요즘 영어 문제를 안 풀어서 실력이 내려가고 있는가 강타요,요즘 영어 문제를 안 풀어서 실력이 내려가고 있는 거 같아요,0,0.796
4,원래 좋아하는 건 잘 찾아 외우는 능력이 잇서서 말이죠,원래 좋아하는 건 잘 찾아 외우는 능력이 이서서 말이죠,원래 좋아하는 것은 잘 찾아 외우는 능력이 있어서 말이죠,0,0.514
...,...,...,...,...,...
95,악마가 한 짓이 아닐가하고 의심하는게요,악마가 한 짓이 아닐 가하고 의심하는데요,악마가 한 짓이 아닐까 하고 의심하는데요,0,0.811
96,알람이 안 떠가지궁 바로 확인할게요,알라임 안 떠가지 궁 바로 확인할게요,알람이 안 떠가지고 바로 확인할게요,0,0.553
97,어떠케 공부하셨는지 여쭤봐두 될까요?,어떠케 공부하셨는지 여쭤봐다 될까요,어떻게 공부하셨는지 여쭤봐도 될까요,0,0.921
98,어떠케 접근하는게 베스트인가요,어떠케 접근하는가 페스트인가요,어떻게 접근하는 게 베스트인가요,0,0.489
